# Install Package

In [35]:
#!pip install selenium
#!brew cask install chromedriver (home dirで実行)
#!pip install webdriver_manager

In [129]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains

import time
import requests
import sys
import datetime
import signal
import os

In [130]:
## 取得したい日付け
get_month   = "li[2]"
get_weekend = "tr[1]"

get_month:
チケットの月を指定する．"li[1]"は今月．来月は"li[2]"とindexが増加

get_weekend:
チケットの第何週の曜日かを指定する．"tr[1]"は月の第1週目．第2週目は"li[2]"とindexが増加

日付けは基本土日のどちらかを取得．<br>
他曜日を取得した場合は，[@id="searchCalendar"] の取得部分の"td[2]"のindexを変更すれば可能．"td[1]"~"td[7]"が月曜日〜日曜日に対応．

In [140]:
def PssportJudgeAndCilck(day, when, notbuy_xpath, click_xpath):
    try:
        NotBuyElement = driver.find_element_by_xpath(notbuy_xpath)
        print(NotBuyElement.text)
        if NotBuyElement.text != '現在、販売していません':
            print("Now on Sales!!! Hurry Up!!!!!")
            driver.find_element_by_xpath(click_xpath).click()
            return None
        else:
            print(NotBuyElement.text)
            return 0
    except:
        print("現在、販売していません")
        return 0

In [141]:
def login():
    #起動
    driver.get("https://reserve.tokyodisneyresort.jp/")
    time.sleep(2)
    
    #ログインクリック
    driver.find_element_by_xpath('//*[@id="header"]/div/ul[2]/li/a').click()
    time.sleep(2)

    #ログイン
    LoginId = driver.find_element_by_xpath('//*[@id="_userId"]')
    LoginId.send_keys("windows.onepeace@gmail.com")
    password = driver.find_element_by_xpath('//*[@id="_password"]')
    password.send_keys("xoqjaM-jynmy6-biskur")
    driver.find_element_by_xpath('//*[@id="_loginConection"]/form/ul/li/button').click()
    time.sleep(2)
    return

In [158]:
def buy_ticket():
    #調査回数をカウントする変数
    RandNum1day = 0
    SeaNum2day = 0

    #販売中であればbreakしてその画面のままにする
    while True:
        time.sleep(.5)
        
        # 土曜日
        # 日付のクリック（ホバーしてクリック、１秒の待機）
        ActionHover = ActionChains(driver)
        ActionHover.move_to_element(driver.find_element_by_xpath(\
        f'//*[@id="searchCalendar"]/div/div/ul/div/div/{get_month}/div/table/tbody/{get_weekend}/td[6]/a')).perform()
        driver.find_element_by_xpath(\
        f'//*[@id="searchCalendar"]/div/div/ul/div/div/{get_month}/div/table/tbody/{get_weekend}/td[6]/a').click()
        time.sleep(1)

        #「自宅でプリントアウト」のクリック
        driver.find_element_by_xpath('//*[@id="searchEticket"]').click()
        time.sleep(1)
        
        
        #青四角クリックができればクリック、できなければ次のアクションへ
        Rand1dayElements = PssportJudgeAndCilck(1, '1day', '//*[@id="searchResultList"]/ul/li[1]/div/p[3]',\
        '//*[@id="searchResultList"]/ul/li[1]/div')
        Rand1030toElements = PssportJudgeAndCilck(1, '10:30-', '//*[@id="searchResultList"]/ul/li[2]/div/p[3]', \
        '//*[@id="searchResultList"]/ul/li[2]/div')
        if Rand1dayElements==None or Rand1030toElements==None:
            print("day1")
            break

        #確認用出力
        RandNum1day += 1
        print(f'チケット売り切れ中  :', RandNum1day, "周目", datetime.datetime.now())
        print(Rand1dayElements, Rand1030toElements)


        time.sleep(2)
        
        
        # 日曜日
        # 日付のクリック
        ActionHover = ActionChains(driver)
        ActionHover.move_to_element(driver.find_element_by_xpath(\
        f'//*[@id="searchCalendar"]/div/div/ul/div/div/{get_month}/div/table/tbody/{get_weekend}/td[7]/a')).perform()
        driver.find_element_by_xpath(\
        f'//*[@id="searchCalendar"]/div/div/ul/div/div/{get_month}/div/table/tbody/{get_weekend}/td[7]/a').click()
        time.sleep(1)


        #「自宅でプリントアウト」のクリック
        driver.find_element_by_xpath('//*[@id="searchEticket"]').click()
        time.sleep(1)

        # もしクリックができればクリックとライン通知
        Sea1dayElements = PssportJudgeAndCilck(2, '1day', '//*[@id="searchResultList"]/ul/li[1]/div/p[3]', \
        '//*[@id="searchResultList"]/ul/li[1]/div')
        Sea1030toElements = PssportJudgeAndCilck(2, '10:30-', '//*[@id="searchResultList"]/ul/li[2]/div/p[3]', \
        '//*[@id="searchResultList"]/ul/li[2]/div')
        if Sea1dayElements == None or Sea1030toElements == None:
            print("day2")
            break

        # 確認用出力
        SeaNum2day += 1
        print('チケット売り切れ中  :', SeaNum2day, "周目", datetime.datetime.now())
        print(Sea1dayElements, Sea1030toElements)

        # 確認用出力総合
        if(SeaNum2day%5 == 0):
            line_ntfy_kakuninn(SeaNum2day*2)
    return True

In [159]:
# Seleniumを起動
from urllib3.util import wait
options = Options()
options.add_argument('--disable-gpu')
options.add_argument('--disable-extensions')
options.add_argument('--proxy-server="direct://"')
options.add_argument('--proxy-bypass-list=*')
options.add_argument('--start-maximized')

driber_path = "/usr/local/bin/chromedriver"
driver = webdriver.Chrome(executable_path=driber_path)
login()


'''
driver.get("https://reserve.tokyodisneyresort.jp/ticket/search/?route=1")
check = buy_ticket()
if check:
    print("yeah")

'''
while True:
    try:
        driver.get("https://reserve.tokyodisneyresort.jp/ticket/search/?route=1")
        check = buy_ticket()
        if check:
            break;
    except:
        time.sleep(3)
        pass


/var/folders/2s/3stzp5kn1wl8tkq0_v9d57940000gp/T/ipykernel_14500/559347726.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driber_path)
/var/folders/2s/3stzp5kn1wl8tkq0_v9d57940000gp/T/ipykernel_14500/4288754393.py:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ActionHover.move_to_element(driver.find_element_by_xpath(\


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="searchCalendar"]/div/div/ul/div/div/li[2]/div/table/tbody/tr[1]/td[6]/a"}
  (Session info: chrome=95.0.4638.54)
Stacktrace:
0   chromedriver                        0x000000010bb4dc99 __gxx_personality_v0 + 579209
1   chromedriver                        0x000000010bad95f3 __gxx_personality_v0 + 102371
2   chromedriver                        0x000000010b69f288 chromedriver + 172680
3   chromedriver                        0x000000010b6d425c chromedriver + 389724
4   chromedriver                        0x000000010b706104 chromedriver + 594180
5   chromedriver                        0x000000010b6f13cd chromedriver + 508877
6   chromedriver                        0x000000010b703eab chromedriver + 585387
7   chromedriver                        0x000000010b6f1693 chromedriver + 509587
8   chromedriver                        0x000000010b6c7b7e chromedriver + 338814
9   chromedriver